In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import os
import math
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score


In [ ]:
df = pd.read_parquet(r"data/pp_joind.parquet", engine="pyarrow")

In [37]:
df

,neid,LOCALCELLID,dateday,DLEARFCN,cellreselpriority_x,qrxlevmin_x,snonintrasearch,thrshservlow,cellreselpriority_y,qoffsetfreq,qrxlevmin_y,threshxhigh,threshxlow,orig_idx,TIM_THROU_USER_PDCP_DL (Kbps),TIM_PRB_UTIL_MEAN_DL (%)
48168,18NLITAG01,1,2023-09-18,100,7,-60,15,1,7,dB14,-58.0,1,0,168,20400.410918,18.284000
48169,18NLITAG01,1,2023-09-18,100,7,-60,15,1,7,dB14,-58.0,1,0,168,24518.369100,14.074000
48170,18NLITAG01,1,2023-09-18,100,7,-60,15,1,7,dB14,-58.0,1,0,168,13574.512948,3.518000
48171,18NLITAG01,1,2023-09-18,100,7,-60,15,1,7,dB14,-58.0,1,0,168,8917.557690,8.392000
48172,18NLITAG01,1,2023-09-18,100,7,-60,15,1,7,dB14,-58.0,1,0,168,11995.291205,4.430000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3905365,SLRJZ602,9,2023-10-01,9410,7,-60,15,1,5,dB12,-58.0,1,0,20403,63681.882353,1.361333
3905366,SLRJZ602,9,2023-10-01,9410,7,-60,15,1,5,dB12,-58.0,1,0,20403,3600.000000,1.341333
3905367,SLRJZ602,9,2023-10-01,9410,7,-60,15,1,5,dB12,-58.0,1,0,20403,26472.000000,1.316000
3905368,SLRJZ602,9,2023-10-01,9410,7,-60,15,1,5,dB12,-58.0,1,0,20403,3960.000000,1.290667


In [ ]:
# Calcular tamanho de cada partição (10% do total)
tamanho_particao = math.ceil(len(df) * 0.1)

num_particao = 1
for i in range(0, len(df), tamanho_particao):
    particao = df.iloc[i:i + tamanho_particao].copy()

    # Normalizar apenas as colunas numéricas
    num_cols = particao.select_dtypes(include="number").columns
    X_scaled = StandardScaler().fit_transform(particao[num_cols])

    # Rodar DBSCAN
    dbscan = DBSCAN(eps=1, min_samples=5)
    labels = dbscan.fit_predict(X_scaled)

    # Adicionar labels na partição
    particao["cluster_dbscan"] = labels

    # Salvar CSV
    nome_arquivo = os.path.join("data", f"particao_{num_particao}.csv")
    particao.to_csv(nome_arquivo, index=False)
    print(f"Partição {num_particao} salva com {len(particao)} linhas")
    num_particao += 1


Partição 1 salva com 119388 linhas
Partição 2 salva com 119388 linhas
Partição 3 salva com 119388 linhas
Partição 4 salva com 119388 linhas
Partição 5 salva com 119388 linhas
Partição 6 salva com 119388 linhas
Partição 7 salva com 119388 linhas
Partição 8 salva com 119388 linhas
Partição 9 salva com 119388 linhas
Partição 10 salva com 119384 linhas


### Clustering 

In [39]:
particoes = [f"data/particao_{i}.csv" for i in range(1, 11)]
resultados = []

for caminho in particoes:
    df_particao = pd.read_csv(caminho, engine="pyarrow")

    if "cluster_dbscan" not in df_particao.columns:
        print(f"⚠ {caminho} não possui coluna 'cluster_dbscan'. Pulando.")
        continue

    # Criar máscara removendo ruído
    mask = df_particao['cluster_dbscan'] != -1
    df_sem_ruido = df_particao[mask]

    # Se não sobrou dado, pula
    if df_sem_ruido.empty:
        resultados.append({"particao": caminho, "silhueta": None})
        df_particao["silhueta"] = None
        df_particao.to_csv(caminho, index=False)
        continue

    # Selecionar colunas numéricas (exceto cluster)
    num_cols = df_sem_ruido.select_dtypes(include='number').columns
    if 'cluster_dbscan' in num_cols:
        num_cols = num_cols.drop('cluster_dbscan')

    # Calcular silhueta se houver mais de um cluster
    if len(set(df_sem_ruido['cluster_dbscan'])) > 1:
        sil_score = silhouette_score(df_sem_ruido[num_cols], df_sem_ruido['cluster_dbscan'])
    else:
        sil_score = None

    resultados.append({"particao": caminho, "silhueta": sil_score})

    # Adiciona a silhueta como nova coluna (mesmo valor para todas as linhas)
    df_particao["silhueta"] = sil_score
    df_particao.to_csv(caminho, index=False)

# Mostrar resultados
for r in resultados:
    print(f"{r['particao']}: Silhueta = {r['silhueta']}")

data/particao_1.csv: Silhueta = -0.011478008089835528
data/particao_2.csv: Silhueta = -0.12714547103865356
data/particao_3.csv: Silhueta = -0.012084382561928743
data/particao_4.csv: Silhueta = -0.00506013832182247
data/particao_5.csv: Silhueta = -0.1141468476806227
data/particao_6.csv: Silhueta = -0.15247894854150276
data/particao_7.csv: Silhueta = -0.1448639099118847
data/particao_8.csv: Silhueta = -0.04876870047618315
data/particao_9.csv: Silhueta = -0.1481749786607902
data/particao_10.csv: Silhueta = -0.18752159335623608


In [ ]:
# Lista das partições
particoes = [f"data/particao_{i}.csv" for i in range(1, 11)]

# Criar pasta results se não existir
os.makedirs("results", exist_ok=True)

for caminho in particoes:
    df = pd.read_csv(caminho, engine="pyarrow")

    if not all(col in df.columns for col in ['cluster_dbscan']):
        print(f"ERROR:  {caminho} não possui pca1, pca2 e cluster_dbscan. Pulando.")
        continue

    plt.figure(figsize=(8, 6))
    plt.scatter(df['TIM_THROU_USER_PDCP_DL (Kbps)'], df['TIM_PRB_UTIL_MEAN_DL (%)'], c=df['cluster_dbscan'], cmap='viridis', s=10)
    plt.title(f'DBScan Throughput vs PRB util - {os.path.basename(caminho)}')
    plt.xlabel('Thoruput DL (Kbps)')
    plt.ylabel('PRB Util Mean DL (%)')
    

    # Salvar gráfico em results
    nome_saida = os.path.join("results", f"{os.path.basename(caminho).replace('.csv', '')}_dbscan.png")
    plt.savefig(nome_saida, dpi=300, bbox_inches="tight")
    plt.close()

    print(f"Gráfico salvo em {nome_saida}")

✅ Gráfico salvo em results\particao_1_dbscan.png
✅ Gráfico salvo em results\particao_2_dbscan.png
✅ Gráfico salvo em results\particao_3_dbscan.png
✅ Gráfico salvo em results\particao_4_dbscan.png
✅ Gráfico salvo em results\particao_5_dbscan.png
✅ Gráfico salvo em results\particao_6_dbscan.png
✅ Gráfico salvo em results\particao_7_dbscan.png
✅ Gráfico salvo em results\particao_8_dbscan.png
✅ Gráfico salvo em results\particao_9_dbscan.png
✅ Gráfico salvo em results\particao_10_dbscan.png
